## Importing Libraries

In [1]:
import requests
import io
import zipfile
import re
import pandas as pd
import numpy as np
import json
import modules.psql as psql
from sqlalchemy import types as altypes

## Postgres Configuration

In [2]:
%run config_psql.ipynb

## Settings Configuration

In [3]:
# Settings configurations

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Initializing parameters

In [4]:
url = "https://cricsheet.org/downloads/recently_played_30_json.zip"
filetype = ".json"

df_meta = pd.DataFrame()
df_match = pd.DataFrame()
df_official = pd.DataFrame()
df_registry = pd.DataFrame()
df_player = pd.DataFrame()
df_innings = pd.DataFrame()
df_deliveries = pd.DataFrame()
df_powerplay = pd.DataFrame()
df_absent_hurt = pd.DataFrame()
df_miscounted_overs = pd.DataFrame()

## Read the downloaded data

In [5]:
response = requests.get(url)

if response.status_code == 200:
    content = response.content
    
    zip_file = zipfile.ZipFile(io.BytesIO(content))
    
    with zip_file.open('README.txt') as f:
        lines = [line.decode('utf-8') for line in f.readlines()]
        pattern = re.compile(r'(\d{4}-\d{2}-\d{2}) - ([^-]+) - ([^-]+) - (\w+) - (\d+) - (.+)')
        ids = [match.group(5) for line in lines if (match := pattern.match(line))]
    f.close()

## Building indivudal DataFrames for different tables

In [6]:
print(len(ids), " files present")
for file in ids:
    with zip_file.open(file+filetype) as jsonfile:
        data = json.load(jsonfile)
        # -----------------------------
        # DataFrame to store - Metadata
        df_meta = pd.concat([df_meta, pd.DataFrame([data["meta"]]).assign(filename=file, filetype=filetype)])
        
        # ----------------------------------
        # DataFrame to store - match details
        df_info = pd.DataFrame([data["info"]])
        df_match_temp = pd.concat([
            pd.json_normalize(df_info['event'], sep='_').add_prefix('event_'),
            pd.DataFrame(df_info[list(set(['balls_per_over','season', 'gender', 'city', 'venue', 'match_type', 'match_type_number', 'overs', 'team_type']) & set(df_info.columns))]),
            df_info['dates'].apply(lambda x: [x[0], x[-1]]).apply(pd.Series).rename(columns={0: 'start_date', 1: 'end_date'}),
            df_info['teams'].apply(lambda x: [x[0], x[1]]).apply(pd.Series).rename(columns={0: 'team_host', 1: 'team_visitor'}),
            pd.json_normalize(df_info['toss'], sep='_').add_prefix('toss_'),
            pd.json_normalize(df_info['outcome'], sep='_').add_prefix('outcome_')
        ], axis=1).assign(match_id = file)
        if 'player_of_match' in df_info.columns:
            df_match_temp['player_of_match'] = df_info['player_of_match'].apply(lambda x: ','.join(x))
            
        df_match = pd.concat([df_match, df_match_temp])
        
        # -----------------------------------
        # DataFrame to store official details
        df_umpire = pd.json_normalize(df_info['officials'], sep = '_')
        umpire_set = set()
        
        for column in df_umpire.columns:
            umpire_set.update(df_umpire[column].explode().dropna())

        df_umpire2 = pd.DataFrame(index=list(umpire_set), columns=df_umpire.columns).fillna(False)
        
        for column in df_umpire.columns:
            df_umpire2[column] = df_umpire2.index.isin(df_umpire[column].explode().dropna())
            
        df_umpire2 = df_umpire2.reset_index().rename(columns={'index': 'name'}).assign(match_id = file)     
        df_official = pd.concat([df_official, df_umpire2])

        # -------------------------------------
        # DataFrame to store - registry details
        df_registry = pd.concat([
            df_registry,
            pd.DataFrame(list(data["info"]["registry"]["people"].items()), columns=['people', 'identifier']).assign(match_id = file)
        ])
        
        # -----------------------------------------
        # DataFrame to store - match player details
        df_player = pd.concat([
            df_player,
            pd.json_normalize(df_info['players']).melt(var_name='team', value_name='player').explode('player').assign(match_id = file)
        ])
        
        # -------------------------------------------------
        # DataFrame to store - innings details
        df_innings = pd.concat([df_innings,
                                pd.json_normalize(data['innings'], sep = '_').drop('overs', axis = 1).assign(match_id = file)])
        
        # -------------------------------------------------
        # DataFrame to store - deliveries ball by ball
        for i in data['innings']:
            index = data['innings'].index(i)
            df_deliveries = pd.concat([df_deliveries,
                                        pd.json_normalize(data['innings'],
                                                          record_path=['overs', 'deliveries'],
                                                          meta=['team', ['overs', 'over']],
                                                          sep='_'
                                                         )
                                        .assign(match_id=file, inning = index+1)
                                       ])
        
        # --------------------------------------
        # DataFrame to store - powerplay details
        for i in data['innings']:
            if 'powerplays' in pd.json_normalize(i).columns:
                index = data['innings'].index(i)
                df_powerplay = pd.concat([df_powerplay, pd.json_normalize(data['innings'][index], record_path = ['powerplays'], meta = ['team'], sep = '_').assign(match_id = file)])

            if 'absent_hurt' in pd.json_normalize(i).columns:
                index = data['innings'].index(i)
                df_absent_hurt = pd.concat([
                    df_absent_hurt,
                    pd.json_normalize(data['innings'][index])[['team','absent_hurt']].explode('absent_hurt').assign(match_id = file)
                ])                

        df_miscounted_overs = pd.concat([df_miscounted_overs,pd.DataFrame([
            {
                "team": inning.get("team", ""),
                "miscounted_over": over_number,
                "balls": over_data.get("balls", ""),
                "umpire": over_data.get("umpire", "")
            }
            for inning in data.get("innings", [])
            for over_number, over_data in inning.get("miscounted_overs", {}).items()
        ]).assign(match_id = file)])
        
    print(file + " executed!")

113  files present
1388200 executed!
1408107 executed!
1373580 executed!
1387599 executed!
1411271 executed!
1411272 executed!
1386100 executed!
1408106 executed!
1411269 executed!
1411270 executed!
1373579 executed!
1386099 executed!
1387598 executed!
1388199 executed!
1411267 executed!
1411268 executed!
1386098 executed!
1411265 executed!
1411266 executed!
1386097 executed!
1406075 executed!
1408105 executed!
1411263 executed!
1411264 executed!
1373578 executed!
1388198 executed!
1406074 executed!
1408104 executed!
1411261 executed!
1411262 executed!
1386095 executed!
1398256 executed!
1386094 executed!
1408103 executed!
1373577 executed!
1395704 executed!
1398255 executed!
1406073 executed!
1388197 executed!
1373576 executed!
1388196 executed!
1389395 executed!
1398254 executed!
1387228 executed!
1389394 executed!
1407107 executed!
1407108 executed!
1407109 executed!
1407104 executed!
1407105 executed!
1407106 executed!
1409206 executed!
1389393 executed!
1391785 executed!
1391786 e

## Adding/Modifying additional fields

In [7]:
match_id_list = ", ".join([f"'{match_id}'" for match_id in ids])

df_meta['created'] = pd.to_datetime(df_meta['created'])

# Merging registry details into match-player details
df_player.reset_index(inplace = True, drop = True)
df_registry.reset_index(inplace = True, drop = True)
df_player.rename(columns = {'player':'name'}, inplace = True)
df_player['player_id'] = df_player.merge(df_registry, how='left', left_on=['match_id', 'name'], right_on=['match_id', 'people'])['identifier']

df_innings.drop(['powerplays','absent_hurt'], axis=1, inplace=True, errors='ignore')
df_innings.drop(df_innings.filter(like='miscounted_overs_').columns, axis=1, inplace=True, errors='ignore')

if not df_absent_hurt.empty:
    df_absent_hurt.reset_index(inplace = True, drop = True)
    df_absent_hurt.rename(columns = {'absent_hurt':'name'}, inplace = True)
    df_absent_hurt['player_id'] = df_absent_hurt.merge(df_registry, how='left', left_on=['match_id', 'name'], right_on=['match_id', 'people'])['identifier']

## Load data into Database

#### 1. Metadata

In [121]:
# Upsert MetaData information
query = psql.upsert(
    engine,
    dataFrame = df_meta,
    table = "meta",
    schema = "dwh",
    pk_col = list(df_meta.columns),
    update_col = list(df_meta.columns))

#### 2. Officials(umpires)

In [108]:
# Load official(umpires) information
with engine.connect() as conn:
    conn.execute(f"DELETE FROM dwh.official WHERE match_id IN ({match_id_list})")
    
count_rows = df_official.to_sql('official', schema='dwh', con=engine, if_exists='append', method='multi', index=False)

with engine.connect() as conn:
    conn.execute("""
        UPDATE dwh.official OF
        SET is_registered = FALSE
        FROM dwh.people P
        WHERE OF.name = P.identifier AND P.identifier IS NULL;
    """)

#### 3. Player-match (players who played a particular match)

In [ ]:
# Load player-match information
with engine.connect() as conn:
    conn.execute(f"DELETE FROM dwh.match_player WHERE match_id IN ({match_id_list})")
    
count_rows = df_player.to_sql('match_player', schema='dwh', con=engine, if_exists='append', method='multi', index=False)

with engine.connect() as conn:
    conn.execute("""
        UPDATE dwh.match_player MP
        SET is_registered = FALSE
        FROM dwh.people P
        WHERE MP.player_id = P.identifier AND P.identifier IS NULL;
    """)

#### 4. Match details

In [ ]:
# Load match information into Stage table
with engine.connect() as conn:
    conn.execute("TRUNCATE TABLE stg.match")

count_rows = df_match.to_sql('match', schema = 'stg', con = engine, if_exists='append', method = 'multi', index = False)

In [ ]:
# Load match information into dwh layer
with engine.connect() as conn:
    conn.execution_options(isolation_level = "AUTOCOMMIT")
    with conn.begin():
        conn.execute("CALL dwh.LoadMatch()")

#### 5. absent hurt details

In [9]:
query = psql.insert_without_duplicate(
    engine,
    dataFrame = df_absent_hurt,
    table = "absent_hurt",
    schema = "dwh",
    conflict_col = list(df_absent_hurt.columns))

#### 6. miscounted overs

In [8]:
query = psql.insert_without_duplicate(
    engine,
    dataFrame = df_miscounted_overs,
    table = "miscounted_over",
    schema = "dwh",
    conflict_col = list(df_miscounted_overs.columns))
query

'No records found.'

#### 7. innings

In [13]:
with engine.connect() as conn:
    conn.execute("TRUNCATE TABLE stg.inning")
    
df_innings.to_sql('inning', schema = 'stg', con = engine, if_exists='append', method = 'multi', index = False)

235

In [14]:
with engine.connect() as conn:
    conn.execution_options(isolation_level = "AUTOCOMMIT")
    with conn.begin():
        conn.execute("CALL dwh.LoadInning()") 

#### 8. powerplay

In [20]:
with engine.connect() as conn:
    conn.execute("TRUNCATE TABLE stg.powerplay")

df_powerplay.to_sql('powerplay', schema = 'stg', con = engine, if_exists='append', method = 'multi', index = False)

276

In [21]:
with engine.connect() as conn:
    conn.execution_options(isolation_level = "AUTOCOMMIT")
    with conn.begin():
        conn.execute("CALL dwh.LoadPowerplay()") 

#### 9. delivery

In [134]:
with engine.connect() as conn:
    conn.execute("TRUNCATE TABLE stg.delivery")

df_deliveries.to_sql('delivery', 
                     schema = 'stg', 
                     con = engine, 
                     if_exists='append', 
                     method = 'multi', 
                     dtype = {
                                 "wickets":altypes.JSON(none_as_null=True)
                             },
                     index = False)

880

In [141]:
df_deliveries[df_deliveries['replacements_role'].notnull()]

,batter,bowler,non_striker,wickets,runs_batter,runs_extras,runs_total,extras_wides,extras_legbyes,runs_non_boundary,team,overs_over,match_id,inning,replacements_match,extras_noballs,review_by,review_umpire,review_batter,review_decision,review_type,extras_byes,review_umpires_call,replacements_role,extras_penalty
267,SFM Devine,Umaima Sohail,AC Kerr,NaN,0,0,0,NaN,NaN,NaN,New Zealand,43,1388199,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'in': 'Umaima Sohail', 'out': 'Nida Dar', 'r...",NaN
267,SFM Devine,Umaima Sohail,AC Kerr,NaN,0,0,0,NaN,NaN,NaN,New Zealand,43,1388199,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'in': 'Umaima Sohail', 'out': 'Nida Dar', 'r...",NaN
139,DP Hughes,CJ Anderson,JR Philippe,NaN,0,0,0,NaN,NaN,NaN,Sydney Sixers,2,1386098,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'in': 'CJ Anderson', 'out': 'RP Meredith', '...",NaN
139,DP Hughes,CJ Anderson,JR Philippe,NaN,0,0,0,NaN,NaN,NaN,Sydney Sixers,2,1386098,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'in': 'CJ Anderson', 'out': 'RP Meredith', '...",NaN
135,J Mbabazi,MD Bimenyimana,K Awino,NaN,1,0,1,NaN,NaN,NaN,Uganda,1,1411264,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'in': 'J Mbabazi', 'out': 'P Alako', 'reason...",NaN
135,J Mbabazi,MD Bimenyimana,K Awino,NaN,1,0,1,NaN,NaN,NaN,Uganda,1,1411264,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'in': 'J Mbabazi', 'out': 'P Alako', 'reason...",NaN
1219,L Haskett,M Labuschagne,CJ Rocchiccioli,NaN,0,0,0,NaN,NaN,NaN,Western Australia,92,1391787,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'in': 'M Labuschagne', 'out': 'MT Steketee',...",NaN
1219,L Haskett,M Labuschagne,CJ Rocchiccioli,NaN,0,0,0,NaN,NaN,NaN,Western Australia,92,1391787,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'in': 'M Labuschagne', 'out': 'MT Steketee',...",NaN
1219,L Haskett,M Labuschagne,CJ Rocchiccioli,NaN,0,0,0,NaN,NaN,NaN,Western Australia,92,1391787,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'in': 'M Labuschagne', 'out': 'MT Steketee',...",NaN
174,RN Patel,SA Okpe,Sukhdeep Singh,NaN,0,0,0,NaN,NaN,NaN,Kenya,8,1407092,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'in': 'Sukhdeep Singh', 'out': 'IA Karim', '...",NaN


In [ ]:
select
	wickets,
	x.player_out,
	x.kind,
	x.fielders,
	f.name
from 
stg.delivery d,
json_to_recordset(d.wickets) as x(
	player_out text,
	kind text,
	fielders json
),
json_to_recordset(x.fielders) as f(
	name text
)